In [1]:
from qir_trans import load
import os
import sys
import tempfile
import textwrap
from qir_trans.translator import Exporter

In [2]:
qir_content = """
    ; ModuleID = 'instruction_error'
    source_filename = "instruction_error.ll"

    %Result = type opaque
    %Qubit = type opaque

    @str_r1 = internal constant [3 x i8] c"r1\\00"

    declare void @my__result_record_output(%Result*, i8*)

    define void @main() #0 {
    entry:
    call void @my__result_record_output(%Result* null, i8* getelementptr inbounds ([3 x i8], [3 x i8]* @str_r1, i32 0, i32 0))
    ret void
    }

    attributes #0 = { "entry_point" "output_labeling_schema" "qir_profiles"="custom" "required_num_qubits"="0" "required_num_results"="0" }

    !llvm.module.flags = !{!0, !1, !2, !3}

    !0 = !{i32 1, !"qir_major_version", i32 1}
    !1 = !{i32 7, !"qir_minor_version", i32 0}
    !2 = !{i32 1, !"dynamic_qubit_management", i1 false}
    !3 = !{i32 1, !"dynamic_result_management", i1 false}
    """

qir_content = """
    ; ModuleID = 'instruction_error'
    source_filename = "instruction_error.ll"

    %Result = type opaque

    declare void @my_test(%Result*, %Result*)

    define void @main() #0 {
    entry:
    call void @my_test(%Result* null, %Result* null)
    ret void
    }

    attributes #0 = { "entry_point" "output_labeling_schema" "qir_profiles"="custom" "required_num_qubits"="0" "required_num_results"="0" }

    !llvm.module.flags = !{!0, !1, !2, !3}

    !0 = !{i32 1, !"qir_major_version", i32 1}
    !1 = !{i32 7, !"qir_minor_version", i32 0}
    !2 = !{i32 1, !"dynamic_qubit_management", i1 false}
    !3 = !{i32 1, !"dynamic_result_management", i1 false}
    """


qir_content = """
        ; ModuleID = 'llvm_inst_error'
        source_filename = "llvm_inst_error.ll"
        %MyStruct = type { i32, float }

        @g_i32     = global i32 42, align 4
        @g_struct  = global %MyStruct { i32 1, float 1.000000e+00 }, align 8
        @g_array   = global [4 x i8] zeroinitializer, align 1

        define void @main() #0 {
        entry:
        %0 = load i32, i32* @g_i32, align 4
        ret void
        }

        attributes #0 = {"entry_point" "output_labeling_schema" "qir_profiles"="custom" "required_num_qubits"="0" "required_num_results"="0" }
    """


# Create a temporary QIR file
with tempfile.NamedTemporaryFile(mode="w", suffix=".ll", delete=True) as temp_file:
    temp_file.write(textwrap.dedent(qir_content))
    temp_file.flush()  # Ensure content is written to disk

    # Load the QIR module
    module = load(temp_file.name)

    # Convert to QASM using the Exporter
    exporter = Exporter()
    qasm_output = exporter.dumps(module)
    print(qasm_output)

Exception: Undefined llvm instruction: load

In [ ]:
# TEST CASE FILE
path = "../../test/resources/qir_test_file/rxx.ll"
# path = "QIR-example/test.ll"
# path = "QIR-example/bell_pair.ll"
# path = "QIR-example/dynamic_allocation.bc"

In [ ]:
mod = load(path)
print(mod)

; ModuleID = '<string>'
source_filename = "bell"

%Qubit = type opaque

define void @main() #0 {
entry:
  call void @__quantum__qis__rxx__body(double 1.000000e+00, %Qubit* null, %Qubit* inttoptr (i64 1 to %Qubit*))
  ret void
}

declare void @__quantum__qis__rxx__body(double, %Qubit*, %Qubit*)

attributes #0 = { "entry_point" "output_labeling_schema" "qir_profiles"="custom" "required_num_qubits"="2" "required_num_results"="0" }

!llvm.module.flags = !{!0, !1, !2, !3}

!0 = !{i32 1, !"qir_major_version", i32 1}
!1 = !{i32 7, !"qir_minor_version", i32 0}
!2 = !{i32 1, !"dynamic_qubit_management", i1 false}
!3 = !{i32 1, !"dynamic_result_management", i1 false}



In [ ]:
print(Exporter().dumps(mod))

OPENQASM 3.0;
include "stdgates.inc";
qubit[2] Qubits;
gate rxx(θ) q0, q1 {
  h q0;
  h q1;
  cx q0, q1;
  rz(θ) q1;
  cx q0, q1;
  h q0;
  h q1;
}
rxx(1.0) Qubits[0], Qubits[1];

